In [27]:
%load_ext autoreload
%autoreload 2

2017-10-24 10:44:40,296 : INFO : Loading dictionaries from C:\WinPython\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\pymorphy2_dicts\data
2017-10-24 10:44:40,342 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
import pandas as pd
from collections import Counter
import functools
import re
import numpy as np
from sklearn.model_selection import train_test_split
import tqdm
import semantics as sem
import make_sentence_model as sen_model
from sklearn.metrics import roc_curve, auc
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import paired_distances
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifierCV, LogisticRegressionCV
import json
import gensim
import pymorphy2
import pickle
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial

In [6]:
@functools.lru_cache(maxsize=2 ** 19)
def uniq_words(text):
    return set(re.findall("\w+", text))

def calculate_idfs(data):
    counter_paragraph = Counter()
    uniq_paragraphs = data['p_bags'].unique()
    for paragraph in uniq_paragraphs:
        set_words = uniq_words(paragraph)
        counter_paragraph.update(set_words)
        
    num_docs = uniq_paragraphs.shape[0]
    idfs = {}
    for word in counter_paragraph:
        idfs[word] = np.log(num_docs / counter_paragraph[word])
    return idfs

In [7]:
with open('data/stop-words-russian.txt', encoding= 'utf-8' ) as f:
    stopwords = f.readlines() 
stoplist = [word.replace('\n','') for word in stopwords]   

In [291]:
dftrain, dftest = pd.read_csv("data/train_task1_latest.csv"), pd.read_csv("data/test_task1_latest.csv")


In [ ]:
train1, test1 = train_test_split(dftrain,test_size=0.3)
train = train1.copy()
test = test1.copy()

In [292]:
train = dftrain.copy()
test = dftest.copy()

In [10]:
w2v_model = sem.load_w2v_model(sem.WORD2VEC_MODEL_FILE)

2017-10-24 09:51:31,367 : INFO : loading projection weights from data/ruwikiruscorpora_0_300_20.bin.gz
2017-10-24 09:51:51,323 : INFO : loaded (392339, 300) matrix from data/ruwikiruscorpora_0_300_20.bin.gz


word2vec model 'data/ruwikiruscorpora_0_300_20.bin.gz' loaded


In [ ]:
#model_p = sen_model.read_paragraphs(pd.concat([train,test]),'paragraph')
#model_q = sen_model.read_paragraphs(pd.concat([train,test]),'question')

In [293]:
bags_p, bags_q =  sen_model.read_paragraphs(train)
bags_p_test, bags_q_test =  sen_model.read_paragraphs(test)


(119398, 5)


100%|████████████████████████████████████████████████████████| 119398/119398 [05:51<00:00, 1071.76it/s]


(74286, 4)


100%|███████████████████████████████████████████████████████████| 74286/74286 [03:27<00:00, 358.10it/s]


In [294]:
for ex, fname in ((bags_p,'bags_p_dumps.json') , (bags_q,'bags_q_dumps.json'),(bags_p_test,'bags_p_test_dumps.json') , (bags_q_test,'bags_q_test_dumps.json')):
     with open(fname, 'w') as f:
        f.write(json.dumps(ex) + '\n')

In [295]:
#bags_p = json.load(open('bags_p_dumps.json'))
#bags_q = json.load(open('bags_q_dumps.json'))
#bags_p_test = json.load(open('bags_p_test_dumps.json'))
#bags_q_test = json.load(open('bags_q_test_dumps.json'))

In [296]:
def get_sims_bags(row, b_p, b_q):
        p_id = row['paragraph_id']
        q_id = row['question_id']
        sims = []
        q_bag = b_q[str(q_id)]['bags'][0]
        
        sims_intersect = [sem.semantic_similarity(p_bag, q_bag, w2v_model) for p_bag in b_p[str(p_id)]['bags']]
        sims = [s[0] for s in sims_intersect]
        #all_intersections = [ins for s in sims_intersect for ins in s[1] if len(s[1]) > 0]
        all_p_bags = [b for bags in b_p[str(p_id)]['bags'] for b in bags]
        
        return np.max(sims), np.mean(sims), ' '.join(all_p_bags), ' '. join(q_bag), ' '.join(list(set(all_intersections)))

In [297]:
%%time
for name, df, b_p, b_q in [('train', train, bags_p, bags_q), ('test', test, bags_p_test, bags_q_test)]:
        print(f'Processing: {name}....')
        results = df.apply(lambda row: get_sims_bags(row, b_p,b_q), axis = 1)
        df.loc[:,'max_sim'] = [r[0] for r in results]
        df.loc[:,'avg_sim'] = [r[1] for r in results]
        df.loc[:,'p_bags'] = [r[2] for r in results]
        df.loc[:,'q_bag'] = [r[3] for r in results]
        df.loc[:,'intersection'] = [r[3] for r in results]
        

Processing: train....
Processing: test....
Wall time: 12min 55s


In [190]:
'''# Old ver code above
%time
for name, df, b_p, b_q in [('train', train, bags_p, bags_q), ('test', test, bags_p_test, bags_q_test)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        p_id = row['paragraph_id']
        q_id = row['question_id']
        sims = []
        q_bag = b_q[str(q_id)]['bags'][0]
        
        #all_p_bags = []
        #all_intersections = []
        #for p_bag in bags_p[p_id]['bags']:
        #    sim, intersection = sem.semantic_similarity(p_bag, q_bag, w2v_model)
        #    sims.append(sim)
        #    if len(intersection) > 0: all_intersections.extend(intersection)
        #    all_p_bags.extend(p_bag)
        
        sims_intersect = [sem.semantic_similarity(p_bag, q_bag, w2v_model) for p_bag in b_p[str(p_id)]['bags']]
        sims = [s[0] for s in sims_intersect]
        all_intersections = [ins for s in sims_intersect for ins in s[1] if len(s[1]) > 0]
        all_p_bags = [b for bags in b_p[str(p_id)]['bags'] for b in bags]
        
        df.loc[index, 'max_sim'] = np.max(sims)
        df.loc[index, 'avg_sim'] = np.mean(sims)
        #df.loc[index, 'variance_sim'] = np.var(sims)
        df.loc[index, 'p_bags'] = ' '.join(all_p_bags)
        df.loc[index, 'q_bag'] = ' '. join(q_bag)
        df.loc[index, 'intersection'] = ' '.join(list(set(all_intersections)))
'''
                

2017-10-25 10:44:41,623 : INFO : Loading dictionaries from C:\WinPython\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\pymorphy2_dicts\data
2017-10-25 10:44:41,684 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


Wall time: 0 ns


build features for train:  25%|████████▎                        | 30106/119398 [12:03<35:25, 42.01it/s]

KeyboardInterrupt: 

build features for train:  25%|████████▎                        | 30106/119398 [12:20<36:35, 40.68it/s]

In [298]:
train.to_csv('data/train_t.csv', encoding = 'utf-8')
test.to_csv('data/test_t.csv', encoding = 'utf-8')

In [300]:
%%time
idfs = calculate_idfs(train)
for name, df in [('train', train), ('test', test)]:
    for index, row in tqdm.tqdm(df.iterrows(), total=df.shape[0], desc="build features for " + name):
        #question = uniq_words(row.q_bags)
        #paragraph = uniq_words(row.p_bag)
        question = set(row.q_bag.split())
        paragraph = set(row.p_bags.split())
        df.loc[index, 'len_paragraph'] = len(paragraph)
        df.loc[index, 'len_question'] = len(question)
        df.loc[index, 'len_intersection'] = len(paragraph & question)
        df.loc[index, 'idf_question'] = np.sum([idfs.get(word, 0.0) for word in question])
        df.loc[index, 'idf_paragraph'] = np.sum([idfs.get(word, 0.0) for word in paragraph])
        df.loc[index, 'idf_intersection'] = np.sum([idfs.get(word, 0.0) for word in paragraph & question])
        df.loc[index, 'idf_intersection2'] = np.sum([idfs.get(word, 0.0) for word in df.loc[index, 'intersection']])

build features for test: 100%|██████████████████████████████████| 74286/74286 [05:26<00:00, 227.86it/s]


Wall time: 15min 30s


In [330]:
#to find question that better fir to another paragraph
def find_max_max_sims_train(row):
    return train[train.question_id == row.question_id].max_sim.max()

train.loc[:,'max_max_sims'] = train.apply(find_max_max_sims_train, axis = 1)
train.loc[:,'max_is_not_max'] = train.max_max_sims != text.max_sim

def find_max_max_sims_test(row):
    return test[test.question_id == row.question_id].max_sim.max()

test.loc[:,'max_max_sims'] = test.apply(find_max_max_sims_test, axis = 1)
test.loc[:,'max_is_not_max'] = test.max_max_sims != text.max_sim

NameError: name 'text' is not defined

In [325]:
columns1 = ['len_paragraph', 'len_question', 'len_intersection', 'idf_question', 'idf_paragraph', 'idf_intersection']
columns2 = ['len_paragraph', 'len_question', 'max_sim', 'avg_sim', 'max_is_not_max']

X_train, X_test = train_test_split(train, test_size=0.3)
clf1 = GradientBoostingClassifier(n_estimators = 400)
model1 = clf1.fit(X_train[columns1], X_train['target'])

scores1 = [roc_auc_score(X_test['target'], predict) for predict in clf1.staged_predict(X_test[columns1])]
best_tree_num1 = np.argmax(scores1) + 1
max_score1 = np.max(scores1)
print("Best Score = {}, at tree number = {}".format(max_score1, best_tree_num1))


clf2 = GradientBoostingClassifier(n_estimators = 300)
model2 = clf2.fit(X_train[columns2], X_train['target'])
scores2 = [roc_auc_score(X_test['target'], predict) for predict in clf2.staged_predict(X_test[columns2])]
best_tree_num2 = np.argmax(scores2) + 1
max_score2 = np.max(scores2)
print("Best Score2= {}, at tree number = {}".format(max_score2, best_tree_num2))


Best Score = 0.9690617902195764, at tree number = 372
Best Score2= 0.9073257466110525, at tree number = 297


In [54]:
'''
clf2 = LogisticRegressionCV(cv =3)
model2 = clf2.fit(X_train[columns2], X_train['target'])
max_score2  = roc_auc_score(X_test['target'], model2.predict(X_test[columns2]))

print("Best Score = {}".format(max_score2))
'''

Best Score = 0.7381004447091776


In [332]:
clf1 = GradientBoostingClassifier(n_estimators = best_tree_num1)
model1 = clf1.fit(X_train[columns1], X_train['target'])

clf1 = GradientBoostingClassifier(n_estimators = best_tree_num2)
model2 = clf1.fit(X_train[columns2], X_train['target'])

predict1 = model1.predict(X_test[columns1])
predict2 = model2.predict(X_test[columns2])

predict = 0.9*predict1 + 0.1*predict2
scores = roc_auc_score(X_test['target'], predict)

print("Score = {}".format(scores))       


Score = 0.9891113738234695


In [333]:
clf1 = GradientBoostingClassifier(n_estimators = best_tree_num1)
model1 = clf1.fit(train[columns1], train['target'])

clf2 = GradientBoostingClassifier(n_estimators = best_tree_num2)
model2 = clf2.fit(train[columns2], train['target'])

predict1 = model1.predict_proba(test[columns1])[:, 1]
predict2 = model2.predict_proba(test[columns2])[:, 1]

predict = 0.9*predict1 + 0.1*predict2

test['prediction'] = predict

#clf = GradientBoostingClassifier(n_estimators = best_tree_num)
#model = clf.fit(train[columns], train['target'])
#test['prediction'] = model.predict_proba(test[columns])[:, 1]
#test['prediction_1'] = model.predict(test[columns])


In [316]:
test[['paragraph_id', 'question_id', 'prediction']].to_csv("prediction.csv", index=False)

In [334]:
cv = CountVectorizer(ngram_range = (2,2))
quest_cv_test = cv.fit_transform(test['question'])
test['max_words_count'] = np.amax(quest_cv_test, axis = 1).toarray()

In [328]:
test.loc[test.max_words_count > 1, 'prediction'] = 0.05
#test['len-quest'] = len(test.question)
#test.loc[test.question.str.len() > 220, 'prediction'] = 0.05

In [280]:
'''
#to find question that better fir to another paragraph
def find_max_max_sims(row):
    return test[test.question_id == row.question_id].max_sim.max()

test.loc[:,'max_max_sims'] = test.apply(find_max_max_sims, axis = 1)
test.loc[test.max_sim != test.max_max_sims,'prediction'] = 0 
'''

In [336]:
test[['paragraph_id', 'question_id', 'prediction']].to_csv("prediction.csv", index=False)